### Modify the paths in the next two cells.

In [7]:
# predicted answer
answer_path = "/home/Arthur/aicup/final/finalsub_revised1202.txt"
# test dataset tsv
tsv_path = "/home/Arthur/aicup/final/opendid_test.tsv"
# post processing
submission = "/home/Arthur/aicup/final/answer.txt"
# country
country_path = '/home/Arthur/aicup/final/country.txt'

In [8]:
countries = {}

with open(country_path, 'r', encoding='utf-8') as f:
  for line in f.readlines():
    words = line.split('\t')
    for w in words:
      if w.strip():
        countries[w.strip('\n')] = words[0]

In [13]:
sub = open(submission, 'w', encoding="utf-8")
tsv = open(tsv_path, 'r', encoding='utf-8')

In [14]:
def detect(private, line):
  A_index = line.find(private[0])
  index = [A_index]
  temp = private[0]
  temp_index = A_index
  #find first phi
  if A_index >= 0:
    for i in range(1, len(private)):
      if index[i-1] > 0:
        temp_index = index[i-1]
      if '\n' in private[i]:
        index.append(line.find(private[i][:-1], temp_index+len(temp)))
        temp = private[i][:-1]
      else:
        index.append(line.find(private[i], temp_index+len(temp)))
        temp = private[i]

  #can't find first phi
  else:
    for i in range(1, len(private)):
      if index[i-1] > 0:
        temp_index = index[i-1]
      if '\n' in private[i]:
        index.append(line.find(private[i][:-1], temp_index+len(temp)))
        temp = private[i][:-1]
      else:
        index.append(line.find(private[i], temp_index+len(temp)))
        temp = private[i]
  #print(private, index)
  return index

In [15]:
sub = open(submission, 'w', encoding="utf-8")
tsv = open(tsv_path, 'r', encoding='utf-8')
#tsv = open(tsv_path, 'r', encoding='utf-8')
phi = ['PATIENT', 'DOCTOR', 'USERNAME', 'PROFESSION', 'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION', 'STREET', 'CITY', 'STATE', 'COUNTRY', 'ZIP', 'LOCATION-OTHER', 'AGE', 'DATE',
      'TIME', 'DURATION', 'SET', 'PHONE', 'FAX', 'EMAIL', 'URL', 'IPADDR', 'SSN', 'MEDICALRECORD', 'HEALTHPLAN', 'ACCOUNT', 'LICENSE', 'VECHICLE', 'DEVICE', 'BIOID', 'IDNUM', 'OTHER']
time = ['DATE', 'TIME', 'DURATION', 'SET']
with open(answer_path, 'r', encoding="utf-8") as answer:
  tsv_line = tsv.readlines()
  max_length = len(tsv_line)
  tsv_index = 0
  #print(tsv_line)
  for line in answer.readlines():
    words = line.split('\t')
    file_name = words[0]
    phi_name = words[1]
    start = words[2]
    end = words[3]
    data = '\t'.join(words[4:])
    #print(line)
    #drop phi not included in rules
    if phi_name not in phi:
      continue

    #print('answer : ' + file_name + ' ' + start + 'tsv:' + tsv_line[tsv_index].split('\t')[0] + ' ' + tsv_line[tsv_index].split('\t')[1])
    #tsv and answer not in the smae file
    while file_name > tsv_line[tsv_index].split('\t')[0]:
      tsv_index += 1
      if tsv_index >= max_length:
        tsv_index -= 1
        break
    while int(tsv_line[tsv_index].split('\t')[1]) <= int(start) and file_name == tsv_line[tsv_index].split('\t')[0]:
      tsv_index += 1
      if tsv_index >= max_length:
        break
    tsv_index -= 1
    '''if file_name != tsv_line[tsv_index].split('\t')[0]:
      print(line)
      print(tsv_line[tsv_index])
      break'''

    #multiple phi in one line
    if '\\n' in data:
      counts = data.split('\\n')
      phi_type, phi_data, normalized = [phi_name], [], []

      if phi_name in time:
        time_split = counts[0].split('\t')
        phi_data.append(time_split[0])
        normalized.append('\t'.join(time_split[1:]).strip())
      elif phi_name == 'COUNTRY':
        content = tsv_line[tsv_index].split('\t')[2]
        country_count = 0
        for country in countries:
          country_count += 1
          if country in content:
            phi_data.append(country)
            break
        #can't fine country
        if country_count >= max_length:
          phi_type = []
      else:
        phi_data.append(counts[0])



      for i in range(1, len(counts)):
        temp_phi = counts[i].split(':')[0]
        if temp_phi not in phi:
          continue
        if temp_phi in time:
          time_split = counts[i].split('\t')
          temp = ':'.join(time_split[0].split(':')[1:])
          normalized.append('\t'.join(time_split[1:]).strip())
        elif temp_phi == 'COUNTRY':
          content = tsv_line[tsv_index].split('\t')[2]
          #print(tsv_line[tsv_index])
          #print(line)
          for country in countries:
            if country in content:
              phi_type.append(temp_phi)
              phi_data.append(countries[country])
              break
        else:
          temp = ':'.join(counts[i].split(':')[1:])
        phi_type.append(temp_phi)
        phi_data.append(temp.strip())

      time_count = 0
      count = 0


      temp_index = tsv_index
      while count < 3 and file_name == tsv_line[tsv_index].split('\t')[0]:
        split_tsv = tsv_line[tsv_index].split('\t')
        line_start = int(split_tsv[1])
        data_tsv = '\t'.join(split_tsv[2:])
        return_index = detect(phi_data, data_tsv)


        #can't find phi in this line
        if len(set(return_index)) == 1 and return_index[0] == -1:
          if int(tsv_line[tsv_index].split('\t')[1]) > int(start) and file_name == tsv_line[tsv_index].split('\t')[0]:
            #print(file_name,int(tsv_line[tsv_index].split('\t')[1]) , int(start))
            count += 1
          tsv_index += 1

        #find in this line
        else:
          T = 0
          '''if '1066' in line:
            T += 1
            print(return_index)
            print(phi_data)'''

          #calculate start and end index
          split_tsv = tsv_line[tsv_index].split('\t')
          data_tsv = '\t'.join(split_tsv[2:])
          line_start = int(split_tsv[1])

          #print(line_start,phi_data)
          for i in range(len(return_index)):
            if return_index[i] < 0:
              continue
            temp = phi_data[i]
            phi_name = phi_type[i]
            index = return_index[i]
            start = line_start + index
            if '\n' in temp:
              end = start + len(temp) -1
              temp = temp[:-1]
            else:
              end = start + len(temp)
            #write into file
            if phi_name in time:
              sub.write('\t'.join([file_name, phi_name, str(start), str(end), temp, normalized[time_count]])+'\n')
              #print('\t'.join([file_name, phi_name, str(start), str(end), temp, normalized[time_count]]))
              if T:
                print('\t'.join([file_name, phi_name, str(start), str(end), temp]))
              time_count += 1
            elif phi_name == 'COUNTRY':
              sub.write('\t'.join([file_name, phi_name, str(start), str(end), countries[temp]])+'\n')
              #print('\t'.join([file_name, phi_name, str(start), str(end), countries[temp]])+'\n')
            else:
              sub.write('\t'.join([file_name, phi_name, str(start), str(end), temp])+'\n')
              if T:
                print('\t'.join([file_name, phi_name, str(start), str(end), temp]))
          tsv_index += 1
          break
      tsv_index = temp_index

    #no problem line
    else:
      if phi_name == 'COUNTRY':
        content = tsv_line[tsv_index].split('\t')[2]
        country_count = 0
        for country in countries:
          country_count += 1
          if country in content:
            print('c: '+country)
            print(content)
            data = countries[country]
            break
        if country_count < len(countries):
          sub.write('\t'.join([file_name, phi_name, str(start), str(end), data]))
          #print('\t'.join([file_name, phi_name, str(start), str(end), data]))
      else:
        #write into file
        sub.write('\t'.join([file_name, phi_name, str(start), str(end), data]))
        #print('\t'.join([file_name, phi_name, str(start), str(end), data]))


In [16]:
sub.close()
tsv.close()

In [23]:
s = "Additional history from Dr BAILEY Bacayo, testicular tumour forty years ago."
s.find('forty years')

60

In [24]:
s = 'forty years'
len(s)

11